In [1]:
import pytorch_lightning as pl
from  data.data_loader import MyDataModule 
from models import OneLayer
from losses.CrossEntropy import CrossEntropy
from losses.Tversky import FocalTverskyLoss
from losses.Dice import DiceLoss
from losses.Focal import FocalLoss
from losses.LogCosh import LogCoshLoss
from losses.Jacard import JacardLoss
from training.PL_train import Main_Loop
from metrics.accuracy import accuracy
import torch.optim as optim
import torchio as tio

In [2]:
train_transformer = tio.Compose([
    tio.RandomMotion(p=0.2),
    tio.RandomBiasField(p=0.3),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.RandomNoise(p=0.5),
    tio.RandomFlip(),
    tio.OneOf({
        tio.RandomAffine(): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
])

val_transformer = tio.Compose([
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
])

In [3]:
size = (48, 60, 48)

In [4]:
model = OneLayer(n_classes=5).cuda()
criterion = JacardLoss().cuda()
data_module = MyDataModule(
    train_transformer=train_transformer,
    val_transformer = val_transformer,
    prepaire=True,
    type_list = ["t1","t2"],
    sample_list = ["t1","t2"]
    )

In [5]:
trainer = pl.Trainer(gpus=1,max_epochs = 100)
main = Main_Loop(model = model,loss= criterion,type_list = ["t1","t2"],scheduler = optim.lr_scheduler.CosineAnnealingLR,scheduler_args = {"T_max":100})
trainer.fit(main, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
100%|██████████| 1251/1251 [15:14<00:00,  1.37it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | OneLayer   | 10    
1 | loss  | JacardLoss | 0     
-------------------------------------
10        Trainable params
0         Non-trainable params
10        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/ludauter/anaconda3/envs/project/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
